In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <torch/torch.h>
namespace nn = torch::nn;
namespace F = torch::nn::functional;

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/nn/modules/loss.h

# MSE Loss (mean squared error)

Creates a criterion that measures the mean squared error (squared L2 norm) between each element in the input x and target y.

In [3]:
int sample_size = 3;
int number_features = 1;

torch::Tensor input = torch::arange(sample_size*number_features).reshape({sample_size,number_features}) * 1.0;

In [4]:
std::cout << input << std::endl;

 0
 1
 2
[ CPUFloatType{3,1} ]


In [5]:
torch::Tensor target = torch::arange(2, sample_size*number_features+2).reshape({sample_size, number_features}) * 1.0;

In [6]:
std::cout << target << std::endl;

 2
 3
 4
[ CPUFloatType{3,1} ]


In [7]:
auto options = nn::MSELossOptions(torch::kMean);

In [8]:
nn::MSELoss mes_loss(options);

In [9]:
torch::Tensor output = mes_loss -> forward(input, target);

In [10]:
std::cout << output << std::endl;

4
[ CPUFloatType{} ]


## understand mse loss

In [11]:
torch::Tensor custom_output = (target - input).square().mean();

In [12]:
std::cout << custom_output << std::endl;

4
[ CPUFloatType{} ]


# L1 Loss
Creates a criterion that measures the mean absolute error (MAE) between each element in the input x and target y.

In [13]:
auto l1_loss_options = nn::L1LossOptions(torch::kMean);

In [14]:
nn::L1Loss l1_loss= nn::L1Loss(l1_loss_options);

In [15]:
output = l1_loss -> forward(input, target);

In [16]:
std::cout << output << std::endl;

2
[ CPUFloatType{} ]


## understand l1 loss

In [17]:
custom_output = (target - input).abs().mean();

In [18]:
std::cout << custom_output << std::endl;

2
[ CPUFloatType{} ]


# NLLLoss
The negative log likelihood loss. It is useful to train a classification problem with C classes.

Input: (N, C), where C = number of classes

Target: (N)

Output: If reduction is 'none', shape (N) Otherwise, scalar.

In [19]:
int sample_size = 3;
int output_size = 4;

torch::Tensor input = torch::arange(sample_size*output_size).reshape({sample_size, output_size}) * 1.0;

In [20]:
std::cout << input << std::endl;

  0   1   2   3
  4   5   6   7
  8   9  10  11
[ CPUFloatType{3,4} ]


In [21]:
torch::Tensor target = torch::tensor({1,0,3});

In [22]:
std::cout << target << std::endl;

 1
 0
 3
[ CPULongType{3} ]


In [24]:
auto nll_loss_option = nn::NLLLossOptions().reduction(torch::kNone);

In [25]:
nn::NLLLoss nll_loss(nll_loss_option);

In [26]:
output = nll_loss -> forward(input, target);

In [27]:
std::cout << output << std::endl;

-1
-4
-11
[ CPUFloatType{3} ]


# understand nll loss
Select specific columns of each row in a torch Tensor

In [36]:
custom_output = (-1) * input.gather(1, target.reshape({-1,1}));

In [37]:
std::cout << custom_output << std::endl;

-1
-4
-11
[ CPUFloatType{3,1} ]


# CrossEntropyLoss

In [41]:
// sample＿size x class_size

torch::Tensor inputs_ts = torch::tensor({{1.0,2.0,3.0},{4.0,5.0,6.0}}); // Only Tensors of floating point and complex dtype can require gradients

// the first sample belongs to the first class, the second sample belongs to the third class
torch::Tensor target_ts = torch::tensor({0,2}, torch::kLong);

In [42]:
std::cout << inputs_ts << std::endl;

 1  2  3
 4  5  6
[ CPUFloatType{2,3} ]


In [43]:
std::cout << target_ts << std::endl;

 0
 2
[ CPULongType{2} ]


In [45]:
auto entropy_loss_option = nn::CrossEntropyLossOptions().reduction(torch::kMean);

In [46]:
nn::CrossEntropyLoss cross_entropy_loss(entropy_loss_option);

In [47]:
output = cross_entropy_loss -> forward(inputs_ts, target_ts);

In [48]:
std::cout << output << std::endl;

1.40761
[ CPUFloatType{} ]


## understand cross entropy loss¶
cross_entropy_loss = softmax + log + nllloss

In [52]:
torch::Tensor custom_cross_entropy(torch::Tensor input, torch::Tensor target){
    //step1: softmax
    torch::Tensor softmax_value = F::softmax(inputs_ts, 1);
    
    //step2: log
    torch::Tensor log_softmax_value = torch::log(softmax_value);
    
    //step3:nllloss
    torch::Tensor output = F::nll_loss(log_softmax_value, target);
    return output;    
}

In [54]:
torch::Tensor custom_output = custom_cross_entropy(inputs_ts, target_ts);

In [55]:
std::cout << custom_output << std::endl;

1.40761
[ CPUFloatType{} ]
